In [43]:
#Importing Modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [44]:
#Loading the Data
data=pd.read_csv("50_Startups.csv")
data.head()

,R&D Spend,Administration,Marketing Spend,State,Profit
0,165349.20,136897.80,471784.10,New York,192261.83
1,162597.70,151377.59,443898.53,California,191792.06
2,153441.51,101145.55,407934.54,Florida,191050.39
3,144372.41,118671.85,383199.62,New York,182901.99
4,142107.34,91391.77,366168.42,Florida,166187.94


In [51]:
pd.DataFrame(data.groupby("State")["Profit"].agg("mean")

UnsupportedFunctionCall: numpy operations are not valid with groupby. Use .groupby(...).mean() instead

In [3]:
#checking for  missing values:
data.isnull().any()

R&D Spend          False
Administration     False
Marketing Spend    False
State              False
Profit             False
dtype: bool

In [4]:
#Searching for categorical columns 
data.select_dtypes(include='object').head()

,State
0,New York
1,California
2,Florida
3,New York
4,Florida


In [5]:
#Uniques values in categorical columns
data.State.unique()

array(['New York', 'California', 'Florida'], dtype=object)

In [6]:
#Splitting dataset into predictors(independent) and target(dependent) variables
x=data.iloc[:,:-1]
y=data.Profit

In [7]:
x.columns

Index(['R&D Spend', 'Administration', 'Marketing Spend', 'State'], dtype='object')

In [8]:
#OneHotEncoding the State column
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct=ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[3])],remainder='passthrough')
x_new=pd.DataFrame(ct.fit_transform(x))


In [11]:
cols=["New York","California","Florida",'R&D Spend', 'Administration', 'Marketing Spend']

In [9]:
x_new.head()

,0,1,2,3,4,5
0,0.0,0.0,1.0,165349.20,136897.80,471784.10
1,1.0,0.0,0.0,162597.70,151377.59,443898.53
2,0.0,1.0,0.0,153441.51,101145.55,407934.54
3,0.0,0.0,1.0,144372.41,118671.85,383199.62
4,0.0,1.0,0.0,142107.34,91391.77,366168.42


In [12]:
x_new.columns=cols

In [13]:
x_new.head()

,New York,California,Florida,R&D Spend,Administration,Marketing Spend
0,0.0,0.0,1.0,165349.20,136897.80,471784.10
1,1.0,0.0,0.0,162597.70,151377.59,443898.53
2,0.0,1.0,0.0,153441.51,101145.55,407934.54
3,0.0,0.0,1.0,144372.41,118671.85,383199.62
4,0.0,1.0,0.0,142107.34,91391.77,366168.42


In [15]:
x=x_new.rename(columns={"Florida":"New York","New York":"California","California":"Florida"})

In [16]:
x.head()

,California,Florida,New York,R&D Spend,Administration,Marketing Spend
0,0.0,0.0,1.0,165349.20,136897.80,471784.10
1,1.0,0.0,0.0,162597.70,151377.59,443898.53
2,0.0,1.0,0.0,153441.51,101145.55,407934.54
3,0.0,0.0,1.0,144372.41,118671.85,383199.62
4,0.0,1.0,0.0,142107.34,91391.77,366168.42


In [18]:
#splitiing into training and testing sets for validation
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=0.7,test_size=0.3,random_state=0)
x_train.head()

,California,Florida,New York,R&D Spend,Administration,Marketing Spend
7,0.0,1.0,0.0,130298.13,145530.06,323876.68
14,0.0,1.0,0.0,119943.24,156547.42,256512.92
45,0.0,0.0,1.0,1000.23,124153.04,1903.93
48,0.0,0.0,1.0,542.05,51743.15,0.00
29,0.0,0.0,1.0,65605.48,153032.06,107138.38


In [19]:
#Definig the model than training it
from sklearn.linear_model import LinearRegression
regressor=LinearRegression()
regressor.fit(x_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [20]:
regressor.coef_

array([-2.55942855e+02,  2.07086137e+02,  4.88567184e+01,  7.90840255e-01,
        3.01968165e-02,  3.10148566e-02])

In [21]:
regressor.intercept_

42659.81372552097

In [28]:
# making predictions on the test data
pred=regressor.predict(x_test)
pred

array([104282.76472171, 132536.88499213, 133910.85007766,  72584.77489416,
       179920.92761889, 114549.31079234,  66444.43261346,  98404.96840123,
       114499.82808601, 169367.50639894,  96522.6253998 ,  88040.67182871,
       110949.99405524,  90419.1897851 , 128020.46250063])

In [23]:
y_test

28    103282.38
11    144259.40
10    146121.95
41     77798.83
2     191050.39
27    105008.31
38     81229.06
31     97483.56
22    110352.25
4     166187.94
33     96778.92
35     96479.51
26    105733.54
34     96712.80
18    124266.90
Name: Profit, dtype: float64

In [33]:
preds=pd.DataFrame({"Predicted Y":pred,"Actual Y":y_test})

In [36]:
len(preds)

15

In [37]:
preds

,Predicted Y,Actual Y
28,104282.764722,103282.38
11,132536.884992,144259.40
10,133910.850078,146121.95
41,72584.774894,77798.83
2,179920.927619,191050.39
27,114549.310792,105008.31
38,66444.432613,81229.06
31,98404.968401,97483.56
22,114499.828086,110352.25
4,169367.506399,166187.94


In [40]:
preds.index=range(15)

In [41]:
preds

,Predicted Y,Actual Y
0,104282.764722,103282.38
1,132536.884992,144259.40
2,133910.850078,146121.95
3,72584.774894,77798.83
4,179920.927619,191050.39
5,114549.310792,105008.31
6,66444.432613,81229.06
7,98404.968401,97483.56
8,114499.828086,110352.25
9,169367.506399,166187.94
